# Question1 
Re-analyse the annual maximum gust wind speeds for (I) the years 1952 to 1998, (II) the years 1952 to 1997, i.e. ignore the high value recorded in 1998. Compare the resulting predictions of design wind speeds for (a) 50 years return period, and (b) 1000 years return period, and comment.
# Question2 
Using the parameter estimation approach in code GB 50009 2012, predict the 10, 20, 50, 100, 200, 500, 1000 years return period design wind speeds for the above two cases, and compare the results with the Gumbel approach.


In [25]:

# 导入必要的库，pandas用于数据处理
import pandas as pd
from io import StringIO # python3中StringIO的位置

# 加载数据
data = """
year,Wind speed m/s
1952,31.4
1953,33.4
1954,29.8
1955,30.3
1956,27.8
1957,30.3
1958,29.3
1959,36.5
1960,29.3
1961,27.3
1962,31.9
1963,28.8
1964,25.2
1965,27.3
1966,23.7
1967,27.8
1968,32.4
1969,27.8
1970,26.2
1971,30.9
1972,31.9
1973,27.3
1974,25.7
1975,32.9
1976,28.3
1977,27.3
1978,28.3
1979,28.3
1980,29.3
1981,27.8
1982,27.8
1983,30.9
1984,26.7
1985,30.3
1986,28.3
1987,30.3
1988,34
1989,28.8
1990,30.3
1991,27.3
1992,27.8
1993,28.8
1994,30.9
1995,26.2
1996,25.7
1997,24.7
1998,42.2
"""

# 将data数据转换为StringIO对象，再读取为DataFrame
df = pd.read_csv(StringIO(data)) 


# Answer


### Question1

In [26]:
import pandas as pd
import numpy as np
from io import StringIO

# 使用矩估计法估计 Gumbel 分布的参数
def estimate_gumbel_params(speed_data):
    """使用矩估计法估计 Gumbel 分布的参数。"""
    mean = np.mean(speed_data)
    std = np.std(speed_data, ddof=1)  # 无偏标准差
    beta = (std * np.sqrt(6)) / np.pi
    mu = mean - 0.5772 * beta  # 欧拉常数
    return mu, beta

# 计算给定重现期 T 的设计风速 (Gumbel 分布)
def design_wind_speed_gumbel(T, mu, beta):
    """计算给定重现期 T 的设计风速 (Gumbel 分布)。"""
    if T <= 1:
        return np.inf  # 处理 T <= 1 的边界情况
    return mu - beta * np.log(-np.log(1 - 1/T))

# 情况 I 和 II 的数据
df_I = df.copy()  # 情况 I: 包含 1998 年
df_II = df[df['year'] < 1998]  # 情况 II: 排除 1998 年

# ----------------------- Gumbel 方法分析 -----------------------

def analyze_gumbel(df, label):
    """执行 Gumbel 分析并打印结果。"""
    mu, beta = estimate_gumbel_params(df['Wind speed m/s'])
    T50 = design_wind_speed_gumbel(50, mu, beta)
    T1000 = design_wind_speed_gumbel(1000, mu, beta)
    print(f"\n{label}:")
    print(f"参数: μ={mu:.2f}, β={beta:.2f}")
    print(f"50年重现期设计风速: {T50:.2f} m/s")
    print(f"1000年重现期设计风速: {T1000:.2f} m/s")

print("问题 1: Gumbel 方法分析")
analyze_gumbel(df_I, "情况 I (包含 1998 年, n=47)")
analyze_gumbel(df_II, "情况 II (排除 1998 年, n=46)")

问题 1: Gumbel 方法分析

情况 I (包含 1998 年, n=47):
参数: μ=27.83, β=2.49
50年重现期设计风速: 37.55 m/s
1000年重现期设计风速: 45.04 m/s

情况 II (排除 1998 年, n=46):
参数: μ=27.82, β=2.01
50年重现期设计风速: 35.67 m/s
1000年重现期设计风速: 41.71 m/s


#### comment

1. Case I (Including 1998 data): Because this high wind speed value is included, the estimated parameters μ (location parameter) and β (scale parameter) are both higher than in Case II. This leads to significantly higher design wind speeds for both the 50-year and 1000-year return periods. This implies that if the 1998 data is considered valid, the design needs to account for higher wind speeds, increasing the structure's safety.

2. Case II (Excluding 1998 data): After excluding the 1998 data, the values of μ and β decrease, and the design wind speeds for the 50-year and 1000-year return periods also decrease accordingly. This suggests that if the 1998 data is considered an outlier or erroneous data, a lower design wind speed can be adopted, thus reducing structural costs.

In [27]:
import pandas as pd
import numpy as np
from io import StringIO

# 划分数据集
df_I = df.copy()  # 包含1998年，n=47
df_II = df[df['year'] < 1998]  # 排除1998年，n=46

# Gumbel方法参数估计（矩估计法）
def estimate_gumbel_params(speed_data):
    mean = np.mean(speed_data)
    std = np.std(speed_data, ddof=1)  # 无偏标准差
    beta = (std * np.sqrt(6)) / np.pi
    mu = mean - 0.5772 * beta
    return mu, beta

# 设计风速计算
def design_wind_speed(T, mu, beta):
    if T <= 1:
        return np.inf
    return mu - beta * np.log(-np.log(1 - 1/T))

# 计算参数和设计风速
# 情况I（包含1998年）
mu_I, beta_I = estimate_gumbel_params(df_I['Wind speed m/s'])
T50_I = design_wind_speed(50, mu_I, beta_I)
T1000_I = design_wind_speed(1000, mu_I, beta_I)

# 情况II（排除1998年）
mu_II, beta_II = estimate_gumbel_params(df_II['Wind speed m/s'])
T50_II = design_wind_speed(50, mu_II, beta_II)
T1000_II = design_wind_speed(1000, mu_II, beta_II)

# 输出结果
print("问题1：Gumbel方法分析")
print("\n情况I（包含1998年，n=47）：")
print(f"参数: μ={mu_I:.2f}, β={beta_I:.2f}")
print(f"50年重现期设计风速: {T50_I:.2f} m/s")
print(f"1000年重现期设计风速: {T1000_I:.2f} m/s")

print("\n情况II（排除1998年，n=46）：")
print(f"参数: μ={mu_II:.2f}, β={beta_II:.2f}")
print(f"50年重现期设计风速: {T50_II:.2f} m/s")
print(f"1000年重现期设计风速: {T1000_II:.2f} m/s")

问题1：Gumbel方法分析

情况I（包含1998年，n=47）：
参数: μ=27.83, β=2.49
50年重现期设计风速: 37.55 m/s
1000年重现期设计风速: 45.04 m/s

情况II（排除1998年，n=46）：
参数: μ=27.82, β=2.01
50年重现期设计风速: 35.67 m/s
1000年重现期设计风速: 41.71 m/s


### Question2

In [28]:
import pandas as pd
import numpy as np
from io import StringIO
from scipy.interpolate import interp1d

# GB规范方法参数估计
TABLE_E32 = np.array([
    [10, 0.9497, 0.4952], [15, 1.02057, 0.5182], [20, 1.06283, 0.52355],
    [25, 1.09145, 0.53086], [30, 1.11238, 0.53622], [35, 1.12847, 0.54034],
    [40, 1.14132, 0.54362], [45, 1.15185, 0.54630], [50, 1.16066, 0.54853],
    [60, 1.17465, 0.55208], [70, 1.18536, 0.55477], [80, 1.19385, 0.55688],
    [90, 1.20649, 0.5586], [100, 1.20649, 0.56002], [250, 1.24292, 0.56878],
    [500, 1.2588, 0.57240], [1000, 1.26851, 0.57450], [np.inf, 1.28255, 0.57722]
])

# 使用 scipy 的线性插值
interpolate_C1 = interp1d(TABLE_E32[:, 0], TABLE_E32[:, 1], kind='linear', fill_value="extrapolate")
interpolate_C2 = interp1d(TABLE_E32[:, 0], TABLE_E32[:, 2], kind='linear', fill_value="extrapolate")

# 估计 GB 50009-2012 方法的参数
def gb_parameters(speed_data):
    n = len(speed_data)
    x_bar = np.mean(speed_data)
    sigma1 = np.std(speed_data, ddof=1)
    C1 = interpolate_C1(n)  # 插值得到的 C1
    C2 = interpolate_C2(n)  # 插值得到的 C2
    alpha = C1 / sigma1
    u = x_bar - C2 / alpha
    return alpha, u

# 计算给定重现期 T 的设计风速
def design_wind_speed_gb(T, alpha, u):
    if T <= 1:
        return np.inf
    return u - (1 / alpha) * np.log(-np.log(1 - 1 / T))

# 情况 I 和 II 的数据
df_I = df.copy()  # 情况 I: 包含 1998 年
df_II = df[df['year'] < 1998]  # 情况 II: 排除 1998 年

# GB 50009-2012 方法分析

def analyze_gb(df, label):
    """执行 GB 方法分析并打印结果。"""
    alpha, u = gb_parameters(df['Wind speed m/s'])
    T_values = [10, 20, 50, 100, 200, 500, 1000]  # 重现期
    results = []
    for T in T_values:
        design_speed = design_wind_speed_gb(T, alpha, u)
        results.append([T, design_speed])
    result_df = pd.DataFrame(results, columns=["Return Period", "GB Method"])

    print(f"\n{label}:")
    print(f"GB 方法参数: α={alpha:.4f}, u={u:.2f}")
    print(result_df.round(2))

print("\n问题 2: GB 50009-2012 方法分析")
analyze_gb(df_I, "情况 I (包含 1998 年, n=47)")
analyze_gb(df_II, "情况 II (排除 1998 年, n=46)")


问题 2: GB 50009-2012 方法分析

情况 I (包含 1998 年, n=47):
GB 方法参数: α=0.3615, u=27.75
   Return Period  GB Method
0             10      33.98
1             20      35.97
2             50      38.55
3            100      40.48
4            200      42.40
5            500      44.94
6           1000      46.86

情况 II (排除 1998 年, n=46):
GB 方法参数: α=0.4475, u=27.76
   Return Period  GB Method
0             10      32.79
1             20      34.40
2             50      36.48
3            100      38.04
4            200      39.60
5            500      41.65
6           1000      43.20
